# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "http://localhost:1984"
os.environ["LANGCHAIN_PROJECT"] = "DEEPLEARNING.AI"

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI, PromptLayerChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
# llm = ChatOpenAI(temperature=0.0)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_2_230707"], temperature=0.0)
memory = ConversationBufferMemory()

In [4]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [5]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 20 Jul 2023 04:09:54 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e985257cc98934a-ICN', 'alt-svc': 'h3=":443"; ma=86400'}.



> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [8]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}

In [10]:
memory = ConversationBufferMemory()

In [11]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [14]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [16]:
from langchain.memory import ConversationBufferWindowMemory

In [17]:
memory = ConversationBufferWindowMemory(k=1)

In [18]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [19]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [20]:
# llm = ChatOpenAI(temperature=0.0)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_2_230707"], temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [21]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [22]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [23]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

In [24]:
print(memory.buffer)

[HumanMessage(content='Hi, my name is Andrew', additional_kwargs={}, example=False), AIMessage(content="Hello Andrew! It's nice to meet you. How can I assist you today?", additional_kwargs={}, example=False), HumanMessage(content='What is 1+1?', additional_kwargs={}, example=False), AIMessage(content='1+1 is equal to 2.', additional_kwargs={}, example=False), HumanMessage(content='What is my name?', additional_kwargs={}, example=False), AIMessage(content="I'm sorry, but I don't have access to personal information.", additional_kwargs={}, example=False)]


## ConversationTokenBufferMemory

In [25]:
# !pip install tiktoken

In [26]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
# llm = ChatOpenAI(temperature=0.0)
llm = PromptLayerChatOpenAI(pl_tags=["langchain_2_230707"], temperature=0.0)

In [27]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [28]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [29]:
from langchain.memory import ConversationSummaryBufferMemory

In [30]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)


In [31]:
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [32]:
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})  # max_token_limit을 초과하여 요약 요청 발생

In [33]:
memory.load_memory_variables({})  

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [34]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [35]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'

In [36]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'}

In [37]:
# create a long string
schedule1 = "오전 8시에 제품 팀과 회의가 있습니다. \
파워포인트 프레젠테이션을 준비해야 합니다. \
오전 9시부터 오후 12시까지 LangChain 프로젝트 작업을 할 시간이 있는데,\
LangChain은 매우 강력한 도구이기 때문에 빠르게 진행될 것입니다. \
정오에는 최신 AI를 이해하기 위해 한 시간 넘게 운전해 온 고객과 함께 이탈리안 레스토랑에서 점심을 먹게 됩니다. \
최신 LLM 데모를 보여드리기 위해 노트북을 지참하는 것을 잊지 마세요."

memory1 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory1.save_context({"input": "Hello"}, {"output": "What's up"})
memory1.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory1.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule1}"})

In [38]:
memory1.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day, and the AI responds with a detailed list of tasks and meetings, including a presentation, project work, and a lunch meeting with a customer. The AI also reminds the human to bring their laptop for a demo.'}

In [39]:
conversation1 = ConversationChain(
    llm=llm, 
    memory = memory1,
    verbose=True
)

In [40]:
conversation1.predict(input="어떤 데모를 보여주면 좋을까요?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day, and the AI responds with a detailed list of tasks and meetings, including a presentation, project work, and a lunch meeting with a customer. The AI also reminds the human to bring their laptop for a demo.
Human: 어떤 데모를 보여주면 좋을까요?
AI:

> Finished chain.


'데모를 보여주기 좋은 것은 여러 가지가 있을 수 있습니다. 예를 들어, 우리의 최신 제품 기능을 소개하는 데모, 고객의 요구에 맞는 맞춤형 솔루션을 보여주는 데모, 또는 우리의 프로젝트 진행 상황을 시각적으로 보여주는 데모 등이 있을 수 있습니다. 어떤 종류의 데모를 보여드릴지 결정하시면, 해당 분야에 대한 자세한 내용을 제공해 드릴 수 있습니다.'

In [41]:
memory1.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day, and the AI responds with a detailed list of tasks and meetings, including a presentation, project work, and a lunch meeting with a customer. The AI also reminds the human to bring their laptop for a demo. The human asks what kind of demo would be good to show, and the AI suggests various options such as showcasing the latest product features, demonstrating customized solutions for customers, or visually presenting the progress of the project. The AI offers to provide more information on the chosen demo once the human decides.'}